In [1]:
!pip install --upgrade huggingface_hub

<h1>BERT Sentiment Analysis on IMDb </h1>

<h3>1. Install Required Libraries </h3>

In [2]:
!pip install -q transformers datasets accelerate scikit-learn torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

<h3>2. Import Libraries</h3>

import torch

In [3]:
import torch

from transformers import (
AutoTokenizer,
AutoModelForSequenceClassification,
Trainer,
TrainingArguments,
DataCollatorWithPadding
)

from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score

2025-10-12 09:42:47.871368: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760262168.252376      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760262168.355090      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


<h3>3. Check GPU</h3>

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device: ", device)

Using device:  cuda


<h3>4. Load IMDb dataset</h3>

In [5]:
dataset = load_dataset("imdb")
print(dataset)

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [6]:
print(dataset["train"])

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})


In [7]:
print(dataset["train"][1])

{'text': '"I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn\'t matter what one\'s political views are because this film can hardly be taken seriously on any level. As for the claim that frontal male nudity is an automatic NC-17, that isn\'t true. I\'ve seen R-rated films with male nudity. Granted, they only offer some fleeting views, but where are the R-rated films with gaping vulvas and flapping labia? Nowhere, because they don\'t exist. The same goes for those crappy cable shows: schlongs swinging in the breeze but not a clitoris in sight. And those pretentious indie movies like The Brown Bunny, in which we\'re treated to the site of Vincent Gallo\'s throbbing johnson, but not a trace of pink visible on Chloe Sevigny. Before crying (or implying) "double-standard" in matters of nudity, the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women: there are no genitals on display when actresses appears nude, 

In [8]:
print(dataset["train"][1]["text"])


"I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn't matter what one's political views are because this film can hardly be taken seriously on any level. As for the claim that frontal male nudity is an automatic NC-17, that isn't true. I've seen R-rated films with male nudity. Granted, they only offer some fleeting views, but where are the R-rated films with gaping vulvas and flapping labia? Nowhere, because they don't exist. The same goes for those crappy cable shows: schlongs swinging in the breeze but not a clitoris in sight. And those pretentious indie movies like The Brown Bunny, in which we're treated to the site of Vincent Gallo's throbbing johnson, but not a trace of pink visible on Chloe Sevigny. Before crying (or implying) "double-standard" in matters of nudity, the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women: there are no genitals on display when actresses appears nude, and the same cann

In [9]:
print(dataset["train"][2]["label"])

0


In [10]:
print(dataset.keys())

dict_keys(['train', 'test', 'unsupervised'])


In [11]:
print(dataset["train"])

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})


In [12]:
print(type(dataset["train"]))

<class 'datasets.arrow_dataset.Dataset'>


In [13]:
print(dataset["train"][2].keys())

dict_keys(['text', 'label'])


In [14]:
print(dataset["train"][2])

{'text': "If only to avoid making this type of film in the future. This film is interesting as an experiment but tells no cogent story.<br /><br />One might feel virtuous for sitting thru it because it touches on so many IMPORTANT issues but it does so without any discernable motive. The viewer comes away with no new perspectives (unless one comes up with one while one's mind wanders, as it will invariably do during this pointless film).<br /><br />One might better spend one's time staring out a window at a tree growing.<br /><br />", 'label': 0}


<h3>5. Load Tokenizer</h3>

In [15]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

<h3>Create Tokenize Text Function</h3>

In [16]:
def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding=False,
        max_length=256
    )


<h3>Apply Tokenization</h3>

In [17]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

<h3>Prepare Data for PyTorch</h3>

In [18]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

<h3>Prepare Data for PyTorch</h3>

In [19]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [20]:
data_collator

DataCollatorWithPadding(tokenizer=BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

<h3>Rename "label" to "labels" (Trainer expects that)</h3>

In [21]:
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

train_dataset = tokenized_dataset["train"]
test_dataset = tokenized_dataset["test"]

In [22]:
train_dataset = tokenized_dataset["train"]
test_dataset = tokenized_dataset["test"]

<h3>Load Pretrained Model</h3>

In [33]:
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<h3>Define Evaluation Metrics</h3>

In [34]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions)
    return {"accuracy": acc, "f1": f1}

<h3>Training Arguments<h3>

In [35]:
training_args = TrainingArguments(
    output_dir="./results",           # Where to save checkpoints
    eval_strategy="epoch",            # Evaluate at the end of each epoch
    save_strategy="epoch",            # Save model at the end of each epoch
    learning_rate=2e-5,               # Typical for BERT fine-tuning
    per_device_train_batch_size=16,   # Batch size per GPU
    per_device_eval_batch_size=16,    # Eval batch size per GPU
    num_train_epochs=2,               # Number of epochs
    weight_decay=0.01,                # L2 regularization
    load_best_model_at_end=True,      # Automatically load best model
    logging_dir="./logs",             # TensorBoard / logging dir
    logging_steps=50,                 # Print logs every 50 steps
    report_to="none",                 # Disable wandb to avoid warnings
    fp16=True,                         # Enable mixed precision for faster GPU training
    dataloader_pin_memory=True         # Speed up data transfer to GPU
)

<h3>Trainer Setup</h3>

In [36]:
trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/tmp/ipykernel_37/3156397598.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


<h3>Fine Tune Model</h3>

In [37]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.228300,0.200876,0.920520,0.919856
2,0.142000,0.220739,0.922840,0.923528


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=1564, training_loss=0.2086804359770187, metrics={'train_runtime': 1735.3528, 'train_samples_per_second': 28.813, 'train_steps_per_second': 0.901, 'total_flos': 6577776384000000.0, 'train_loss': 0.2086804359770187, 'epoch': 2.0})

In [38]:
metrics = trainer.evaluate()
print(metrics)


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.2008763998746872, 'eval_accuracy': 0.92052, 'eval_f1': 0.9198564110837737, 'eval_runtime': 212.8758, 'eval_samples_per_second': 117.439, 'eval_steps_per_second': 3.674, 'epoch': 2.0}


In [39]:
model.save_pretrained("./fine_tuned_bert_imdb")
tokenizer.save_pretrained("./fine_tuned_bert_imdb")

('./fine_tuned_bert_imdb/tokenizer_config.json',
 './fine_tuned_bert_imdb/special_tokens_map.json',
 './fine_tuned_bert_imdb/vocab.txt',
 './fine_tuned_bert_imdb/added_tokens.json',
 './fine_tuned_bert_imdb/tokenizer.json')

In [48]:
sample_text = "shit"
inputs = tokenizer(sample_text, return_tensors="pt", truncation=True, padding=True).to(device)
outputs = model(**inputs)
pred = torch.argmax(outputs.logits, dim=1).item()
label = "Positive" if pred == 1 else "Negative"
print(f"Review: {sample_text}\nPrediction: {label}")

Review: shit
Prediction: Negative
